# Clustering NBA Players with t-SNE

Having viewed a [TED talk](https://www.youtube.com/watch?v=E-gpSQQe3w8&t=784s) on the evolution of the notion of positions in basketball, I was curious how this applied to the current state of the NBA.

This notebook explores the current NBA but also looks back at previous decades to illuminate some interesting developments in the modernization of the sport.

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from functools import reduce
from plotly import __version__
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)
np.set_printoptions(suppress=True)

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Helper methods

In [2]:
def display_all(df: object) -> object:
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

## Load data

The first [dataset](https://www.kaggle.com/drgilermo/nba-players-stats) contains season stats for individual players since 1950. Let's take a look.

In [3]:
PATH = "data/nba-players-stats/"

In [4]:
df_raw = pd.read_csv(f'{PATH}Seasons_Stats.csv', low_memory=False)

The dataset contains the standard metrics (points, rebounds, steal, blocks, etc.), but it also conatins more advanced metrics (TS%, WS, USG%, etc.)

In [5]:
display_all(df_raw.tail().T)

,24686,24687,24688,24689,24690
Unnamed: 0,24686,24687,24688,24689,24690
Year,2017,2017,2017,2017,2017
Player,Cody Zeller,Tyler Zeller,Stephen Zimmerman,Paul Zipser,Ivica Zubac
Pos,PF,C,C,SF,C
Age,24,27,20,22,19
Tm,CHO,BOS,ORL,CHI,LAL
G,62,51,19,44,38
GS,58,5,0,18,11
MP,1725,525,108,843,609
PER,16.7,13,7.3,6.9,17


Here is a summary of the data.

In [30]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Year,24624,NaN,NaN,NaN,1992.59,17.4296,1950,1981,1996,2007,2017
Player,24624,3921,Eddie Johnson,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pos,24624,23,PF,4966,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,24616,NaN,NaN,NaN,26.6644,3.84189,18,24,26,29,44
Tm,24624,69,TOT,2123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G,24624,NaN,NaN,NaN,50.8371,26.4962,1,27,58,75,88
GS,18233,NaN,NaN,NaN,23.5934,28.6324,0,0,8,45,83
MP,24138,NaN,NaN,NaN,1209.72,941.147,0,340,1053,1971,3882
PER,24101,NaN,NaN,NaN,12.4791,6.03901,-90.6,9.8,12.7,15.6,129.1
TS%,24538,NaN,NaN,NaN,0.493001,0.094469,0,0.458,0.506,0.544,1.136


## Pre-Processing

Before moving forward, some pre-processing is necessary (mainly just removing empty and unnecessary rows/columns).

In [6]:
df_raw.drop(df_raw.columns[0], axis=1, inplace=True)
df_raw.dropna(axis=0, how='all', inplace=True)
df_raw.dropna(axis=1, how='all', inplace=True)
df_raw.fillna(0, inplace=True);

Another detail we have to consider is that the dataset only contains the full set of statistics for players from 1980 onwards. For this initial analysis, only data from 1980 onwards will be considered.

In [7]:
df_modern = df_raw[df_raw.Year >= 1980]

Next, let's take a look at the positions of players.

In [13]:
df_modern.Pos.unique()

array(['C', 'PF', 'PG', 'SG', 'SF', 'SG-PG', 'SF-SG', 'SG-SF', 'C-PF',
       'PF-C', 'SF-PF', 'PG-SG', 'PF-SF', 'PG-SF', 'SG-PF', 'C-SF'],
      dtype=object)

The array contains the five traditional positions; however, they also contain hybrids like PG-SG. These are fine, but the dataset also contains entries with SG-PG. For this analyis, we'll treat them as the same position.

## Player clustering

To begin, let's cluster players in *df_modern* according to the following five metrics:
- PTS
- AST
- REB
- BLK
- STL

In [9]:
tsne = TSNE(n_components=3, init='pca', random_state=0)
Y = tsne.fit_transform(df_modern.loc[df_modern.Year > 2015,
                                     ['PTS', 'AST', 'TRB', 'BLK', 'STL']])

In [11]:
players = df_modern.loc[df_modern.Year > 2015, 
                        ['Player']].astype(str).values.tolist()
players = reduce(lambda z, y :z + y, players)

data = [go.Scatter3d(
    x = Y[:,0],
    y = Y[:,1],
    z = Y[:,2],
    mode = 'markers',
    marker = list(
        color = cmap[]),
    hovertext = players)]

layout = go.Layout(
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter3d: 'color'

    Valid properties:
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on plot.ly for  customdata .
        error_x
            plotly.graph_objects.scatter3d.ErrorX instance or dict
            with compatible properties
        error_y
            plotly.graph_objects.scatter3d.ErrorY instance or dict
            with compatible properties
        error_z
            plotly.graph_objects.scatter3d.ErrorZ instance or dict
            with compatible properties
        hoverinfo
    

### Scoring style

In [128]:
cats = ['TRB%', 'AST%', 'STL%', 'BLK%']
tsne = TSNE(n_components=3, init='pca', random_state=0)
Y = tsne.fit_transform(df_modern.loc[df_modern.Year == 2017, cats])

In [129]:
players = df_modern.loc[df_modern.Year == 2017, 
                        ['Player']].astype(str).values.tolist()
players = reduce(lambda z, y :z + y, players)

data = [go.Scatter3d(
    x = Y[:,0],
    y = Y[:,1],
    z = Y[:,2],
    mode = 'markers',
    hovertext = players)]

layout = go.Layout(
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## My implementation of t-SNE

I implement the t-SNE algorithm in Python with the help of PyTorch to compute the gradient of the cost function.

<img src="img/tsne-algorithm.png" alt="term-document matrix" style="width: 90%"/>

In [ ]:
import torch as torch


## Resources

[Visualizing Data usinag t-SNE](http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)